In [ ]:
!ls /dev/ttyUSB0

In [ ]:
import serial
import time

class E220:
    LORA_SPEED15625_SF5_BW125KHZ  = 0
    LORA_SPEED9375_SF6_BW125KHZ   = 4
    LORA_SPEED5469_SF7_BW125KHZ   = 8
    LORA_SPEED3125_SF8_BW125KHZ   = 12
    LORA_SPEED1758_SF9_BW125KHZ   = 16
    LORA_SPEED31250_SF5_BW250KHZ  = 1
    LORA_SPEED18750_SF6_BW250KHZ  = 5
    LORA_SPEED10938_SF7_BW250KHZ  = 9
    LORA_SPEED6520_SF7_BW250KHZ   = 13
    LORA_SPEED3516_SF8_BW250KHZ   = 17
    LORA_SPEED1953_SF9_BW250KHZ   = 21
    LORA_SPEED62500_SF5_BW500KHZ  = 2
    LORA_SPEED37500_SF6_BW500KHZ  = 6
    LORA_SPEED21875_SF7_BW500KHZ  = 10
    LORA_SPEED12500_SF8_BW500KHZ  = 14
    LORA_SPEED7031_SF9_BW500KHZ   = 18
    LORA_SPEED3906_SF9_BW500KHZ   = 22
    LORA_SPEED2148_SF9_BW500KHZ   = 26
    
    def __init__(self, port='/dev/ttyUSB0', baudrate=9600):
        self.serial = serial.Serial(port, baudrate, timeout=1)
        self.params = {}

    def sendCommand(self, message, read_length):
        self.serial.write(bytearray(message))
        time.sleep(1)  # Give the device some time to respond
        response = self.serial.read(read_length)
        return response

    def getConfigure(self):
        response = self.sendCommand([0xC1, 0x00, 0x09], 12)
        if len(response) == 12 and response[0] == 0xC1 and response[1] == 0x00 and response[2] == 0x09:
            self.params = {
                'ADDH': response[3],
                'ADDL': response[4],
                'REG0': response[5],
                'REG1': response[6],
                'REG2': response[7],
                'REG3': response[8],
                'CRYPT_H': response[9],
                'CRYPT_L': response[10],
                'Version': response[11]
            }
            return True
        else:
            print("Invalid response or no response")
            return False
    
    def setRSSI(self, rssi):
        # 先ず7ビット目を0にする
        self.params['REG3'] &= ~(1 << 7)
        
        # 次に、rssiの最下位ビットを7ビット目にセットする
        self.params['REG3'] |= ((rssi & 1) << 7)

    def getRSSI(self):
        # 7ビット目の値を取得して返す
        return (self.params['REG3'] >> 7) & 0x01
    
    def setSendMode(self, mode):
        # 先ず6ビット目を0にする
        self.params['REG3'] &= ~(1 << 6)
        
        # 次に、modeの最下位ビットを6ビット目にセットする
        self.params['REG3'] |= ((mode & 1) << 6)

    def getSendMode(self):
        # 6ビット目の値を取得して返す
        return (self.params['REG3'] >> 6) & 0x01
    
    def setLoRaChannel(self, channel):
        self.params['REG2'] = channel

    def getLoRaChannel(self):
        return self.params.get('REG2', None)  # Returns None if 'REG2' not set

    def setLoRaRate(self, rate):
        # Clear the lower 5 bits
        self.params['REG0'] &= 0xE0  # This is equivalent to 11100000 in binary
        
        # Set the lower 5 bits with the rate's lower 5 bits
        self.params['REG0'] |= (rate & 0x1F)  # 0x1F is equivalent to 00011111 in binary

    def getLoRaRate(self):
        # Return the lower 5 bits
        return self.params['REG0'] & 0x1F
    
    def getADDH(self):
        return self.params['ADDH']

    def setADDH(self, value):
        self.params['ADDH'] = value

    def getADDL(self):
        return self.params['ADDL']

    def setADDL(self, value):
        self.params['ADDL'] = value

    def getLoRaAddress(self):
        return (self.params['ADDH'] << 8) + self.params['ADDL']
    
    def setConfigure(self):
        # Prepare the message with command header
        message = [0xC0, 0x00, 0x08]
        # Convert parameter values to a list of bytes, excluding 'Version'
        params_bytes = list(self.params.values())[:-1]
        # Combine message with parameters
        message_plus_params = message + params_bytes
        # Send command and wait for the response
        response = self.sendCommand(message_plus_params, 11)
        # Check if the first byte of the response is 0xC1, indicating success
        if response[0] == 0xC1:
            return True
        else:
            return False

    def displayConfigure(self):
        if self.params:
            print("Configuration Received:")
            for key, value in self.params.items():
                print(f"{key}: {value}")
        else:
            print("No configuration to display.")

if __name__ == "__main__":
    device = E220()
    if device.getConfigure():
        print("Original Configuration:")
        device.displayConfigure()

        # Set a new LoRa channel and apply the change
        new_channel = 1  # Example new channel
        set_rssi = False
        set_mode = True
        address = 0x009  # 仮のLoRaアドレス
        
        print(f"\nSetting new LoRa channel to: {new_channel}")
        device.setLoRaChannel(new_channel)
        
        print(f"Setting new setRSSI to: {set_rssi}")        
        device.setRSSI(set_rssi)
        
        print(f"Setting new setSendMode to: {set_mode}")        
        device.setSendMode(set_mode)
        print(f"Setting new setLoRaRate to: {E220.LORA_SPEED62500_SF5_BW500KHZ}")   
        device.setLoRaRate(E220.LORA_SPEED62500_SF5_BW500KHZ)
        
        # Addressを設定
        device.setADDH(address >> 8)  # addressの高位バイト
        device.setADDL(address & 0xFF)  # addressの低位バイト
        print(f"Setting new setLoRaAddress to: {0x001}")
        
        if device.setConfigure():
            print("\nConfiguration updated successfully.")

            # Re-fetch and display the configuration to confirm change
            if device.getConfigure():
                print("\nUpdated Configuration:")
                device.displayConfigure()
            else:
                print("Failed to update configuration.")
        else:
            print("Failed to set new configuration.")
            
        # Example to re-fetch and display the LoRa Channel to confirm change
        updated_channel = device.getLoRaChannel()
        print(f"\nUpdated LoRa Channel: {updated_channel}")
        updated_rssi = device.getRSSI()
        print(f"Updated LoRa RSSI: {updated_rssi}")
        updated_sendmode = device.getSendMode()
        print(f"Updated LoRa SendMode: {updated_sendmode}")
        updated_lorarate = device.getLoRaRate()
        print(f"Updated LoRa Lora Rate: {updated_lorarate}")
        updated_address = device.getLoRaAddress()
        print(f"Updated LoRa Lora Address: {updated_address}")
    
    else:
        print("Failed to get original configuration.")


In [ ]:
import serial
import time

# シリアルポートの設定
port = '/dev/ttyUSB0'  # シリアルポートのデバイス名
baudrate = 9600        # ボーレート（デバイスに合わせて設定）
timeout = 1            # タイムアウトの秒数

# シリアルポートを開く
with serial.Serial(port, baudrate, timeout=timeout) as ser:
    print(f"Listening on {port}...")
    
    try:
        while True:
            if ser.in_waiting > 0:
                data = ser.readline()  # データをバイトとして読み取り
                print(f"Received (hex): {data}")
            time.sleep(0.1)  # CPUの使用を抑えるために少し待つ
    except KeyboardInterrupt:
        print("Stopped by user")

In [ ]:
import serial
import time

# シリアルポートの設定
port = '/dev/ttyUSB0'  # シリアルポートのデバイス名
baudrate = 9600        # ボーレート（デバイスに合わせて設定）
timeout = 3            # タイムアウトの秒数

ser = serial.Serial(port, baudrate, timeout=timeout)

def send_data(send_address, send_ch, data):
    global ser
    data_size = len(data)
    message = bytearray(data_size + 4)  # plus 4 for address and ch, plus 1 for delimiter
    message[0] = (send_address >> 8) & 0xFF  # high byte of send_address
    message[1] = send_address & 0xFF  # low byte of send_address
    message[2] = send_ch

    message[3:data_size+3] = data.encode('utf-8')  # assuming data is a str object

    # add delimiter
    message[data_size+3] = ord('\n')

    # Write message
    ser.write(message)

    # Wait for data to become available
    timeout = time.time() + 1  # 1 second timeout
    while ser.in_waiting <= 0 and time.time() < timeout:
        time.sleep(0.01)  # Wait a bit for data to arrive

    # Read response (adjust the size of the read as needed)
    ser.flush()
    if ser.in_waiting > 0:
        response = ser.read(1)
        return response[0]
    else:
        print("No response received.")
        return None


In [ ]:
response = send_data(0x0b, 1, "Hello")
print(response)